In [ ]:
%run ./models.py

In [ ]:
var_pollution = models.Variable("Pollution", ["yes", "no"], "chance")
var_smoker = models.Variable("Smoker", ["yes", "no"], "chance")
var_cancer = models.Variable("Cancer", ["yes", "no"], "chance")
var_xray = models.Variable("Xray", ["yes", "no"], "chance")
var_dyspnoea = models.Variable("Dyspnoea", ["yes", "no"], "chance")

variables = (
    var_pollution,
    var_smoker,
    var_cancer,
    var_xray,
    var_dyspnoea,
)

arcs = (
    models.Arc(var_pollution, var_cancer),
    models.Arc(var_smoker, var_cancer),
    models.Arc(var_cancer, var_xray),
    models.Arc(var_cancer, var_dyspnoea),
)

cpd_pollution = models.CPD(
    variable=var_pollution,
    parents=None,
    table=[[0.9], [0.1]],
    )
cpd_smoker = models.CPD(
    variable=var_smoker,
    parents=None,
    table=[[0.3], [0.7]],
    )
cpd_cancer = models.CPD(
    variable=var_cancer,
    parents=[var_smoker, var_pollution],
    table=[[0.03, 0.05, 0.001, 0.02], [0.97, 0.95, 0.999, 0.98]],
)
cpd_xray = models.CPD(
    variable=var_xray,
    parents=[var_cancer],
    table=[[0.9, 0.2], [0.1, 0.8]],
)
cpd_dyspnoea = models.CPD(
    variable=var_dyspnoea,
    parents=[var_cancer],
    table=[[0.65, 0.3], [0.35, 0.7]],
)

cdps = (
    cpd_pollution,
    cpd_smoker,
    cpd_cancer,
    cpd_xray,
    cpd_dyspnoea,
)

In [ ]:
pgm_model = models.PGM(name="Cancer_model", variables=variables, arcs=arcs, cdps=cdps)

In [ ]:
pgm_model.print_nodes()
pgm_model.print_arcs()
pgm_model.print_variables()
pgm_model.print_potentials()

In [ ]:
pgm_model.model.check_model()

In [ ]:
pgm_model.draw_graph()

In [ ]:
gum_model = models.GUM(name="Cancer_model", variables=variables, arcs=arcs, cdps=cdps)

In [ ]:
gum_model.print_nodes()
gum_model.print_arcs()
gum_model.print_variables()
gum_model.print_potentials()

In [ ]:
gum_model.draw_graph()

In [ ]:
print(pgm_model.inference("Xray"))
print(pgm_model.inference("Dyspnoea"))

In [ ]:
print(gum_model.inference("Xray"))
print(gum_model.inference("Dyspnoea"))